In [1]:
import torch
import torch.nn as nn
import math
from config import *

In [2]:
class BlockResNet(nn.Module):
    def __init__(self, channels: list, kernel_sizes: list, strides: list, paddings: list) -> None:
        super().__init__()
        '''
        input:
            channels = [number of input channels, number of output channels conv1, number of output channels conv2,...]
            len kernel_size = len channels - 1
            len paddings = len kernel_size = len channels - 1
            stride: passed into the first conv layers
        '''
        self.relu = nn.ReLU(inplace=True)
        self.block = nn.Sequential(
            *[nn.Sequential(nn.Conv2d(in_channels= in_channels, 
                      out_channels= out_channels, 
                      kernel_size= kernel_size,
                      stride= stride,
                      padding= padding), nn.BatchNorm2d(out_channels), self.relu if i < len(channels) - 2 else nn.Identity())
            for i, (in_channels, out_channels, stride, kernel_size, padding) in enumerate(zip(channels[:-1], channels[1:], strides + [1]*len(channels), kernel_sizes, paddings))]
        )
        self.shortcut = nn.Identity()
        
        if set(strides) != {1}:
            in_channels = channels[0]
            out_channels = channels[-1]
            stride = math.prod(strides)
            self.shortcut = nn.Conv2d(in_channels= in_channels, 
                      out_channels= out_channels, 
                      kernel_size= 1,
                      stride= stride)

    def forward(self, X):
        out = self.block(X)
        out += self.shortcut(X)
        out = self.relu(out)
        return out

def test_BlockResNet():
    X = torch.Tensor(4,64,56,56)
    config = {
        'channels': [64,128,128],
        'kernel_sizes': [3,3],
        'strides': [2],
        'paddings': [1,1]
    }
    model = BlockResNet(**config)
    out = model(X)
    print(out.shape)
    print(model)
# test_BlockResNet()

In [3]:
class LayerResNet(nn.Module):
    def __init__(self, in_channels_layer: int, out_channels_block: list, kernel_sizes: int, paddings: list, num_blocks: int) -> None:
        super().__init__()
        first_block = BlockResNet(channels= [in_channels_layer] + out_channels_block, kernel_sizes= kernel_sizes, strides= [2],paddings= paddings)
        second_block = BlockResNet(channels= [out_channels_block[-1]] + out_channels_block, kernel_sizes= kernel_sizes, strides= [1],paddings= paddings)
        layer = [first_block] + [second_block]*(num_blocks-1)
        self.layer = nn.Sequential(
            *layer
        )
    def forward(self, X):
        out = self.layer(X)
        return out
    
def test_LayerResNet():
    X = torch.Tensor(4,64,28,28)
    config = {
        'in_channels_layer': 64,
        'out_channels_block': [128,128],
        'kernel_sizes': [3,3],
        'num_blocks': 2,
        'paddings': [1,1]
    }
    model = LayerResNet(**config)
    out = model(X)
    print(out.shape)
    print(model)
# test_LayerResNet()

In [4]:
class ResNet(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        '''
        Description: Each ResNet is made up of 5 layers, each layer is made up of blocks of a certain number
               In each block, there are different conv classes, which are skipped connection
               At each conv starting each layer, stride = 2
        '''
        # append conv1, maxpool
        layer = [nn.Conv2d(**config['conv1']), nn.MaxPool2d(**config['maxpool'])]

        # append conv_x: 2 -> 5
        for i in range(2, 6): 
            key = f'conv{i}_x'
            layer.append(LayerResNet(**config[key]))

        # append average pool
        layer.append(nn.AdaptiveAvgPool2d(1))

        # append flatten
        layer.append(nn.Flatten())

        # append 1000-d fc
        layer.append(nn.Linear(**config['1000-d fc']))

        self.model = nn.Sequential(
            *layer
        )
        self.layer = layer
    def forward(self, X):
        out = self.model(X)
        return out

In [5]:
resNet18 = ResNet(config_resNet18)
X = torch.Tensor(4,3,224,224)
out = resNet18(X)
print(out.shape)
print(resNet18)

torch.Size([4, 1000])
ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): LayerResNet(
      (layer): Sequential(
        (0): BlockResNet(
          (relu): ReLU(inplace=True)
          (block): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): Identity()
            )
          )
          (shortcut): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
        )
        (1): BlockResNet(
    

In [6]:
resNet34 = ResNet(config_resNet34)
out = resNet34(X)
print(out.shape)
print(resNet34)

torch.Size([4, 1000])
ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): LayerResNet(
      (layer): Sequential(
        (0): BlockResNet(
          (relu): ReLU(inplace=True)
          (block): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): Identity()
            )
          )
          (shortcut): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
        )
        (1): BlockResNet(
    

In [7]:
resNet50 = ResNet(config_resNet50)
out = resNet50(X)
print(out.shape)
print(resNet50)

torch.Size([4, 1000])
ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): LayerResNet(
      (layer): Sequential(
        (0): BlockResNet(
          (relu): ReLU(inplace=True)
          (block): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (2): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
              (1): BatchNorm2d(25

In [8]:
resNet101 = ResNet(config_resNet101)
out = resNet101(X)
print(out.shape)
print(resNet101)

torch.Size([4, 1000])
ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): LayerResNet(
      (layer): Sequential(
        (0): BlockResNet(
          (relu): ReLU(inplace=True)
          (block): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (2): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
              (1): BatchNorm2d(25

In [9]:
resNet152 = ResNet(config_resNet152)
out = resNet152(X)
print(out.shape)
print(resNet152)

torch.Size([4, 1000])
ResNet(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): LayerResNet(
      (layer): Sequential(
        (0): BlockResNet(
          (relu): ReLU(inplace=True)
          (block): Sequential(
            (0): Sequential(
              (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (1): Sequential(
              (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (2): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
              (1): BatchNorm2d(25